In [28]:
import pandas as pd
import re
import spacy

from adept.config import DATA_DIR, TRAINING_DIR, PROCESSED_DATA_DIR
from adept.preprocess import Preprocess
from adept.postprocess import Postproccess
import spacy
from spacy.language import Language
from spacy.tokens import Doc
from adept.components.registry import create_sentencizer_component
from adept.components.numeric.range import NumericRange
from adept.utils.helpers import token_get_ent

In [2]:
# nlp = spacy.load('en_core_web_trf')

nlp = spacy.load(TRAINING_DIR / 'adept')

FFFF


In [3]:
nlp.pipeline

[('sentencizer',
  <adept.components.sentencizer.SentencizerComponent at 0x146a06350>),
 ('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x14673fe80>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1450b4880>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1467812a0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1467c4c80>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1467813f0>)]

In [13]:
from spacy.language import Language
from spacy.matcher import Matcher
from spacy.tokens import Span


@Language.factory("numeric_range")
def create_sentencizer_component(nlp: Language, name: str):
    return NumericRange(nlp)


class NumericRange: 
    
    """
    
    Match on MEASUREMENT, CARDINAL and CARDINAL containing range characters (, -, +)
    
    Parses the range into a dict with keys: lower, from, to, upper
    

    Returns:
        doc: doc object with range, is_range extension
    """
    
    name = 'numeric_range'
    
    re_range = re.compile('((?P<lower>[0-9.\/]+)\-?\))?\s?(?P<from>[0-9.\/]+)[\(\)\s]?\-[\(\)\s]?(?P<to>[0-9.\/]+)\s?(\(\-?(?P<upper>[0-9.\/]+))?')
    
    def __init__(self, nlp):
        self.matcher = Matcher(nlp.vocab) 
        # Matcher for (, ) or -, contained within QUANTITY or CARDINAL entity type   
        for ent_type in ['MEASUREMENT', 'DIMENSION', 'CARDINAL', 'VOLUME']:
            self.matcher.add('RANGE', [[{"ENT_TYPE": ent_type}, {"LOWER": {"REGEX": r'^-|\(\)$'}}]])        
                
        Span.set_extension("numeric_range", default=[], force=True)
        # Token.set_extension("is_numeric_range", default=False, force=True)
    
    def __call__(self, doc):

        return call(doc, self.matcher)

ValueError: [E004] Can't set up pipeline component: a factory for 'numeric_range' already exists. Existing factory: <function create_sentencizer_component at 0x10f040040>. New factory: <function create_sentencizer_component at 0x10ef7bc70>

In [44]:
re_range = re.compile('((?P<lower>[0-9.\/]+)\-?\))?\s?(?P<from>[0-9.\/]+)[\(\)\s]?\-[\(\)\s]?(?P<to>[0-9.\/]+)\s?(\(\-?(?P<upper>[0-9.\/]+))?')




In [6]:
nlp.add_pipe("numeric_range", after="ner")

In [45]:
text = 'Rhizomes creeping. Pseudostems 20--40 cm. Leaves sessile except for 2 apical ones; ligule brownish, 1--5 cm, sparsely appressed villous; leaf blade obovate or oblong, 15--27 × 8--10 cm, adaxially glabrous, abaxially appressed yellow villous along midvein, base cuneate, apex acute. Spikes 8--14 × 4--6 cm; rachis appressed yellow villous; bracts rusty, ovate-lanceolate, 3--4.6 × 1.2--2 cm, membranous, appressed yellow villous, 3-flowered; bracteoles tubular, 2.7--3 cm, appressed yellow villous. Flowers white, slightly fragrant. Calyx brownish, 3.3--3.5 cm, slightly split on 1 side, appressed yellow villous. Corolla tube ca. 4 cm, glabrous; lobes linear, 2.5--3 cm × 3--4 mm. Lateral staminodes oblanceolate, ca. 2.3 cm × 7 mm. Labellum broadly ovate, ca. 2.2 × 1.9 cm, apically slightly 2-lobed or 2-cleft for ca. 1/2 its length. Filament white, equaling labellum; anther 9 +- mm. Ovary 5 +- mm, appressed yellow villous. Stigma ciliate. Fl. Feb.'

In [46]:
preprocess = Preprocess()
text = preprocess(text)          
doc = nlp(text)

{'lower': None, 'from': '20', 'to': '40', 'upper': None}
{'lower': None, 'from': '1', 'to': '5', 'upper': None}
{'lower': None, 'from': '15', 'to': '27', 'upper': None}
{'lower': None, 'from': '8', 'to': '10', 'upper': None}
{'lower': None, 'from': '15', 'to': '27', 'upper': None}
{'lower': None, 'from': '8', 'to': '10', 'upper': None}
{'lower': None, 'from': '8', 'to': '14', 'upper': None}
{'lower': None, 'from': '4', 'to': '6', 'upper': None}
{'lower': None, 'from': '8', 'to': '14', 'upper': None}
{'lower': None, 'from': '4', 'to': '6', 'upper': None}
{'lower': None, 'from': '3', 'to': '4.6', 'upper': None}
{'lower': None, 'from': '1.2', 'to': '2', 'upper': None}
{'lower': None, 'from': '3', 'to': '4.6', 'upper': None}
{'lower': None, 'from': '1.2', 'to': '2', 'upper': None}
{'lower': None, 'from': '2.7', 'to': '3', 'upper': None}
{'lower': None, 'from': '3.3', 'to': '3.5', 'upper': None}
{'lower': None, 'from': '2.5', 'to': '3', 'upper': None}
{'lower': None, 'from': '3', 'to': '4',

In [30]:
df = pd.read_csv(PROCESSED_DATA_DIR / 'af4b0c64764555e29e5f5a7847683c2f.descriptions.csv')

In [36]:


for i, text in enumerate(df['description']):
    preprocess = Preprocess()
    text = preprocess(text)          
    doc = nlp(text)
    if (i % 10) == 0:
        print(i)
    # break

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830


20-40 cm
{'lower': None, 'from': '20', 'to': '40', 'upper': None}
1-5 cm
{'lower': None, 'from': '1', 'to': '5', 'upper': None}
15-27 x 8-10 cm
{'lower': None, 'from': '15', 'to': '27', 'upper': None}
{'lower': None, 'from': '8', 'to': '10', 'upper': None}
15-27 x 8-10 cm
{'lower': None, 'from': '15', 'to': '27', 'upper': None}
{'lower': None, 'from': '8', 'to': '10', 'upper': None}
8-14 x 4-6 cm
{'lower': None, 'from': '8', 'to': '14', 'upper': None}
{'lower': None, 'from': '4', 'to': '6', 'upper': None}
8-14 x 4-6 cm
{'lower': None, 'from': '8', 'to': '14', 'upper': None}
{'lower': None, 'from': '4', 'to': '6', 'upper': None}
3-4.6 x 1.2-2 cm
{'lower': None, 'from': '3', 'to': '4.6', 'upper': None}
{'lower': None, 'from': '1.2', 'to': '2', 'upper': None}
3-4.6 x 1.2-2 cm
{'lower': None, 'from': '3', 'to': '4.6', 'upper': None}
{'lower': None, 'from': '1.2', 'to': '2', 'upper': None}
2.7-3 cm
{'lower': None, 'from': '2.7', 'to': '3', 'upper': None}
3.3-3.5 cm
{'lower': None, 'from': '

In [8]:
for ent in doc.sents:
    print(sent)

Rhizomes creeping.
Pseudostems 20-40 cm.
Leaves sessile except for 2 apical ones;
ligule brownish, 1-5 cm, sparsely appressed villous;
leaf blade obovate or oblong, 15-27 x 8-10 cm, adaxially glabrous, abaxially appressed yellow villous along midvein, base cuneate, apex acute.
Spikes 8-14 x 4-6 cm;
rachis appressed yellow villous;
bracts rusty, ovate-lanceolate, 3-4.6 x 1.2-2 cm, membranous, appressed yellow villous, 3-flowered;
bracteoles tubular, 2.7-3 cm, appressed yellow villous.
Flowers white, slightly fragrant.
Calyx brownish, 3.3-3.5 cm, slightly split on 1 side, appressed yellow villous.
Corolla tube ca. 4 cm, glabrous;
lobes linear, 2.5-3 cm x 3-4 mm.
Lateral staminodes oblanceolate, ca. 2.3 cm x 7 mm.
Labellum broadly ovate, ca. 2.2 x 1.9 cm, apically slightly 2-lobed or 2-cleft for ca. 0.5 its length.
Filament white, equaling labellum;
anther ca. 9 mm.
Ovary ca. 5 mm, appressed yellow villous.
Stigma ciliate.
Flower.
Feb.


In [11]:
for ent in doc.ents:
    print(ent.label_)

MEASUREMENT
CARDINAL
MEASUREMENT
DIMENSION
DIMENSION
DIMENSION
NUMERIC_CADJ
MEASUREMENT
MEASUREMENT
CARDINAL
MEASUREMENT
DIMENSION
DIMENSION
DIMENSION
NUMERIC_CADJ
NUMERIC_CADJ
CARDINAL
MEASUREMENT
MEASUREMENT


In [27]:
df.shape

(52091, 73)

In [29]:
df = pd.read_csv(DATA_DIR / '94ed2343-b94a-46dc-bb24-7cc2b080efb2_f5af5f212e1fe3fde9c1c3f736ea60e721480a92' / '05ff2255-c38a-40c9-b657-4ccb55ab2feb.csv')

/var/folders/qc/qfn2fhqj089dgly0lzbhrjrr0000gp/T/ipykernel_57300/4217007686.py:1: DtypeWarning: Columns (14,16,41,42,49,50,51,52,55,56,75,78,79,84,89,91,92,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_DIR / '94ed2343-b94a-46dc-bb24-7cc2b080efb2_f5af5f212e1fe3fde9c1c3f736ea60e721480a92' / '05ff2255-c38a-40c9-b657-4ccb55ab2feb.csv')


In [33]:
countries = [
    'India',
    'Nepal',
    'Sri Lanka',
    'Pakistan',
    'Bhutan',
    'Bangladesh',
    'Maldives',
    'Seychelles'
]


subset = df[df['country'].str.contains('|'.join(countries), na=False, regex = True, flags=re.IGNORECASE)]

In [34]:
subset.shape

(2936, 113)